In [22]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import compose, pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn import compose, pipeline
from sklearn.model_selection import KFold
from typing import Optional
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import roc_auc_score, make_scorer, r2_score
from pandas import DataFrame, Series
import numpy as np
kf = KFold(n_splits=5, shuffle=True, random_state=42)
import xgboost as xgb
from xgboost import XGBRegressor
import optuna
from optuna import Trial
from optuna import create_study
from sklearn import compose
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, Ridge, LinearRegression, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
#website help from 
#https://medium.com/@walter_sperat/using-optuna-with-sklearn-the-right-way-part-1-6b4ad0ab2451

In [2]:
df = pd.read_csv('df_normal_quality.csv', index_col=0)
y = df.SalePrice
X = df.drop(['PID', 'SalePrice'], axis =1).copy()

In [33]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
#from category_encoders import WOEEncoder

def instantiate_ordinal_encoder(trial: Trial)-> OrdinalEncoder:
    params = {
        'handle_unknown': "use_encoded_value", 
        'unknown_value': -1
    }
    
    return OrdinalEncoder(**params)

def instantiate_onehot_encoder(trial: Trial)-> OneHotEncoder:
    params = {
        'handle_unknown': 'ignore',
        'drop': trial.suggest_categorical('drop', [None, 'first'])
    }
    
    return OneHotEncoder(**params)
    
Encoder = (
    OrdinalEncoder |
    OneHotEncoder 
    )

def instantiate_encoder (trial : Trial) -> Encoder:
    encoding_method = trial.suggest_categorical(
        'encoding_method', ['ordinal', 'onehot'])
    if encoding_method =='ordinal':
        encoder = instantiate_ordinal_encoder(trial)
    elif encoding_method =='onehot':
        encoder = instantiate_onehot_encoder(trial)
    
    return encoder

from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler

Scaler = (
  StandardScaler |
  MinMaxScaler |
  MaxAbsScaler |
  RobustScaler
)

def instantiate_scaler(trial : Trial) -> Scaler:
    method = trial.suggest_categorical(
    'scaling_method', ['standard', 'minmax', 'maxabs', 'robust']
    )
    if method=='standard':
        scaler = StandardScaler()
    elif method=='minmax':
        scaler = MinMaxScaler()
    elif method=='maxabs':
        scaler = MaxAbsScaler()
    elif method=='robust':
        scaler = RobustScaler()
        
    return scaler

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

def instantiate_processor(trial : Trial, 
                          numerical_columns : list[str], 
                          categorical_columns : list[str]) -> ColumnTransformer:
    
    numerical_pipeline = instantiate_scaler(trial)
    categorical_pipeline = instantiate_encoder(trial)
   # numerical_pipeline = StandardScaler()
    #categorical_pipeline = OneHotEncoder(handle_unknown="ignore", drop='first')
    
#     numerical_pipeline = instantiate_numerical_pipeline(trial)
#     categorical_pipeline = instantiate_categorical_pipeline(trial)
    
    processor = ColumnTransformer([
        ('numerical_pipeline', numerical_pipeline, numerical_columns),
        ('categorical_pipeline', categorical_pipeline, categorical_columns)
    ])
    return processor

def instantiate_processor_ordinal(trial : Trial, 
                          numerical_columns : list[str], 
                          categorical_columns : list[str]) -> ColumnTransformer:
    
    numerical_pipeline = instantiate_scaler(trial)
    categorical_pipeline = instantiate_ordinal_encoder(trial)

    processor = ColumnTransformer([
        ('numerical_pipeline', numerical_pipeline, numerical_columns),
        ('categorical_pipeline', categorical_pipeline, categorical_columns)
    ])
    return processor

# Lasso

In [44]:
def instantiate_lasso(trial : Trial) -> Lasso:
    params = {
        "alpha": trial.suggest_float("alpha", .00001, .001, log=True)
    }

    return Lasso(**params)

def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_lasso(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_lasso(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [45]:
from optuna import create_study

study_lasso = create_study(study_name='optimization', direction='maximize')

#study_lasso.optimize(lambda trial: objective_lasso(trial, X, y), n_trials=200)

[I 2024-07-11 09:50:20,023] A new study created in memory with name: optimization


In [11]:
print(study_lasso.best_value)
print(study_lasso.best_params)

0.9508124989217249
{'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.00030733929099045147}


In [13]:
trial_number = []
score = []
parameters = []
optuna_dict = {}
trials = range(200)

for trial in trials:
    trial_number.append(study_lasso.get_trials()[trial].number)
    score.append(study_lasso.get_trials()[trial].value)
    parameters.append(list(study_lasso.get_trials()[trial].params.items()))

optuna_dict['Trial'] = trial_number
optuna_dict['Score'] = score
optuna_dict['Parameters'] = parameters

optuna_lasso = pd.DataFrame.from_dict(optuna_dict)
optuna_lasso

,Trial,Score,Parameters
0,0,0.946736,"[(scaling_method, maxabs), (encoding_method, o..."
1,1,0.932313,"[(scaling_method, maxabs), (encoding_method, o..."
2,2,0.941093,"[(scaling_method, robust), (encoding_method, o..."
3,3,0.948149,"[(scaling_method, maxabs), (encoding_method, o..."
4,4,0.947131,"[(scaling_method, robust), (encoding_method, o..."
...,...,...,...
195,195,0.950399,"[(scaling_method, minmax), (encoding_method, o..."
196,196,0.950790,"[(scaling_method, robust), (encoding_method, o..."
197,197,0.950787,"[(scaling_method, robust), (encoding_method, o..."
198,198,0.950707,"[(scaling_method, robust), (encoding_method, o..."


In [67]:
import plotly.express as px


fig = px.scatter(optuna_lasso,
                 x="Trial", 
                 y="Score", 
                 hover_data=['Parameters'],
                labels = {'Trial': 'Optuna Trial', 'Score': 'Score (R-Squared)'},
                title = 'Lasso tuned with Optuna after 200 trials  <br><sup> Best Score: 0.9508</sup>')

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)

fig.show()

# fig.write_image("images/optuna_lasso.png")

# Ridge

In [41]:
def instantiate_ridge(trial : Trial) -> Ridge:
    params = {
        "alpha": trial.suggest_float("alpha", 1, 100, log=True)
    }

    return Ridge(**params)


def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_ridge(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_ridge(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [43]:
from optuna import create_study

study_ridge = create_study(study_name='optimization', direction='maximize')

#study_ridge.optimize(lambda trial: objective_ridge(trial, X, y), n_trials=20)

[I 2024-07-11 09:50:20,011] A new study created in memory with name: optimization


In [17]:
print(study_ridge.best_value)
print(study_ridge.best_params)

0.9496764273611051
{'scaling_method': 'standard', 'encoding_method': 'onehot', 'drop': None, 'alpha': 24.817075515507273}


# EN

In [39]:
def instantiate_en(trial : Trial) -> ElasticNet:
    params = {
        "alpha": trial.suggest_float("alpha", .00001, .001, log=True),
        'l1_ratio': trial.suggest_float('l1_ratio', .1, .7, log = True)
    }

    return ElasticNet(**params)



def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_en(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_en(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [40]:
from optuna import create_study

study_en = create_study(study_name='optimization', direction='maximize')

#study_en.optimize(lambda trial: objective_en(trial, X, y), n_trials=200)

[I 2024-07-11 09:50:19,990] A new study created in memory with name: optimization


In [66]:
# after adding in the L1 value
print(study_en.best_value)
print(study_en.best_params)

0.9508214083737423
{'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0006921322276640134, 'l1_ratio': 0.4415555448483296}


In [29]:
print(study_en.best_value)
print(study_en.best_params)

0.950820633330468
{'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0006178621404820522}


# XGB

In [18]:
def instantiate_xgb(trial : Trial) -> XGBRegressor:
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1, log=True),
        "max_depth": trial.suggest_int("max_depth", 4, 8),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 10, 20)
    }

    return XGBRegressor(**params)

def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_xgb(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_xgb(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [ ]:
from optuna import create_study

study_xgb = create_study(study_name='optimization', direction='maximize')

study_xgb.optimize(lambda trial: objective_xgb(trial, X, y), n_trials=500)

[I 2024-07-16 06:34:01,375] A new study created in memory with name: optimization
[I 2024-07-16 06:34:29,624] Trial 0 finished with value: 0.8576131936296146 and parameters: {'scaling_method': 'minmax', 'learning_rate': 0.04729666006448587, 'loss': 'square', 'n_estimators': 634}. Best is trial 0 with value: 0.8576131936296146.
[I 2024-07-16 06:35:04,288] Trial 1 finished with value: 0.8728219403993756 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.1784712400736335, 'loss': 'exponential', 'n_estimators': 987}. Best is trial 1 with value: 0.8728219403993756.
[I 2024-07-16 06:35:27,271] Trial 2 finished with value: 0.757696746969885 and parameters: {'scaling_method': 'minmax', 'learning_rate': 0.002005894820528472, 'loss': 'square', 'n_estimators': 458}. Best is trial 1 with value: 0.8728219403993756.
[I 2024-07-16 06:36:11,215] Trial 3 finished with value: 0.833354960289751 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.015288270638649756, 'loss': 'exp

[I 2024-07-16 06:49:13,808] Trial 33 finished with value: 0.8703897301040697 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.2348480242381404, 'loss': 'square', 'n_estimators': 957}. Best is trial 11 with value: 0.8775115335127343.
[I 2024-07-16 06:49:39,602] Trial 34 finished with value: 0.87353548800037 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.5289054164112338, 'loss': 'square', 'n_estimators': 878}. Best is trial 11 with value: 0.8775115335127343.
[I 2024-07-16 06:50:03,812] Trial 35 finished with value: 0.8785468108860259 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.9845009908869966, 'loss': 'exponential', 'n_estimators': 816}. Best is trial 35 with value: 0.8785468108860259.
[I 2024-07-16 06:50:36,496] Trial 36 finished with value: 0.8547102815224142 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.0392451385177838, 'loss': 'exponential', 'n_estimators': 715}. Best is trial 35 with value: 0.87854681088602

[I 2024-07-16 07:04:31,388] Trial 66 finished with value: 0.8756922921001846 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.706727875333234, 'loss': 'linear', 'n_estimators': 806}. Best is trial 35 with value: 0.8785468108860259.
[I 2024-07-16 07:05:02,931] Trial 67 finished with value: 0.7518377761192298 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.00042188992779902375, 'loss': 'linear', 'n_estimators': 617}. Best is trial 35 with value: 0.8785468108860259.
[I 2024-07-16 07:05:26,808] Trial 68 finished with value: 0.8765906567072961 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.9543936777022748, 'loss': 'linear', 'n_estimators': 756}. Best is trial 35 with value: 0.8785468108860259.
[I 2024-07-16 07:05:48,427] Trial 69 finished with value: 0.8751550331738682 and parameters: {'scaling_method': 'minmax', 'learning_rate': 0.39034774254268545, 'loss': 'linear', 'n_estimators': 656}. Best is trial 35 with value: 0.8785468108860259.


[I 2024-07-16 07:19:17,965] Trial 99 finished with value: 0.8771028887609511 and parameters: {'scaling_method': 'minmax', 'learning_rate': 0.6232013288441041, 'loss': 'linear', 'n_estimators': 888}. Best is trial 78 with value: 0.8801458176072536.
[I 2024-07-16 07:19:42,523] Trial 100 finished with value: 0.8784001995295035 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.989354175847342, 'loss': 'exponential', 'n_estimators': 860}. Best is trial 78 with value: 0.8801458176072536.
[I 2024-07-16 07:20:05,181] Trial 101 finished with value: 0.8772629685962381 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.8145622691278651, 'loss': 'exponential', 'n_estimators': 773}. Best is trial 78 with value: 0.8801458176072536.
[I 2024-07-16 07:20:30,175] Trial 102 finished with value: 0.8779694271825675 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.97372397848622, 'loss': 'exponential', 'n_estimators': 872}. Best is trial 78 with value: 0.88014581760

[I 2024-07-16 07:34:18,364] Trial 132 finished with value: 0.8772241920798166 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.8096781759158752, 'loss': 'exponential', 'n_estimators': 869}. Best is trial 78 with value: 0.8801458176072536.
[I 2024-07-16 07:34:41,660] Trial 133 finished with value: 0.8775377447663718 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.9902300723886783, 'loss': 'linear', 'n_estimators': 780}. Best is trial 78 with value: 0.8801458176072536.
[I 2024-07-16 07:35:05,231] Trial 134 finished with value: 0.876904035898644 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.6900361647168772, 'loss': 'exponential', 'n_estimators': 741}. Best is trial 78 with value: 0.8801458176072536.
[I 2024-07-16 07:35:29,853] Trial 135 finished with value: 0.8776012737491531 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.8116809002834358, 'loss': 'linear', 'n_estimators': 847}. Best is trial 78 with value: 0.88014

[I 2024-07-16 07:48:02,406] Trial 165 finished with value: 0.8778750809156053 and parameters: {'scaling_method': 'minmax', 'learning_rate': 0.8302477435371738, 'loss': 'exponential', 'n_estimators': 959}. Best is trial 78 with value: 0.8801458176072536.
[I 2024-07-16 07:48:29,958] Trial 166 finished with value: 0.8781691908097489 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.9888553929542855, 'loss': 'exponential', 'n_estimators': 856}. Best is trial 78 with value: 0.8801458176072536.
[I 2024-07-16 07:48:58,356] Trial 167 finished with value: 0.8761737714676894 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.6262849459764945, 'loss': 'linear', 'n_estimators': 878}. Best is trial 78 with value: 0.8801458176072536.
[I 2024-07-16 07:49:23,913] Trial 168 finished with value: 0.8765589767309143 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.7326812321065647, 'loss': 'exponential', 'n_estimators': 816}. Best is trial 78 with value: 0.8

[I 2024-07-16 08:02:42,030] Trial 198 finished with value: 0.8755156768667808 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.7079961964356589, 'loss': 'exponential', 'n_estimators': 422}. Best is trial 78 with value: 0.8801458176072536.
[I 2024-07-16 08:03:22,144] Trial 199 finished with value: 0.7437786534888381 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.00017646541825304187, 'loss': 'exponential', 'n_estimators': 814}. Best is trial 78 with value: 0.8801458176072536.
[I 2024-07-16 08:03:50,371] Trial 200 finished with value: 0.8765788157164582 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.5885677431205414, 'loss': 'linear', 'n_estimators': 918}. Best is trial 78 with value: 0.8801458176072536.
[I 2024-07-16 08:04:17,584] Trial 201 finished with value: 0.8788713238572076 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.9895655271596935, 'loss': 'linear', 'n_estimators': 939}. Best is trial 78 with value: 0.

[I 2024-07-16 08:17:51,780] Trial 231 finished with value: 0.8779077421440243 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.8422999316556743, 'loss': 'exponential', 'n_estimators': 935}. Best is trial 78 with value: 0.8801458176072536.
[I 2024-07-16 08:18:20,013] Trial 232 finished with value: 0.8777251658007914 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.7244607337584574, 'loss': 'exponential', 'n_estimators': 925}. Best is trial 78 with value: 0.8801458176072536.
[I 2024-07-16 08:18:48,162] Trial 233 finished with value: 0.8789825072084592 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.8384006191150075, 'loss': 'exponential', 'n_estimators': 894}. Best is trial 78 with value: 0.8801458176072536.
[I 2024-07-16 08:19:15,104] Trial 234 finished with value: 0.8789070216553447 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.8433449735038682, 'loss': 'exponential', 'n_estimators': 892}. Best is trial 78 with value: 0.88

[I 2024-07-16 08:34:01,751] Trial 264 finished with value: 0.8769335336168442 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.8629659250224222, 'loss': 'linear', 'n_estimators': 902}. Best is trial 78 with value: 0.8801458176072536.
[I 2024-07-16 08:34:38,952] Trial 265 finished with value: 0.8672831654747666 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.06610037627052544, 'loss': 'linear', 'n_estimators': 949}. Best is trial 78 with value: 0.8801458176072536.
[I 2024-07-16 08:35:04,937] Trial 266 finished with value: 0.8771198455806761 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.9984811152777835, 'loss': 'linear', 'n_estimators': 922}. Best is trial 78 with value: 0.8801458176072536.
[I 2024-07-16 08:35:31,014] Trial 267 finished with value: 0.8767668866090859 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.582894424273159, 'loss': 'linear', 'n_estimators': 894}. Best is trial 78 with value: 0.8801458176072536.
[I 2

[I 2024-07-16 08:49:33,216] Trial 297 finished with value: 0.8770386813216631 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.6122621983636913, 'loss': 'exponential', 'n_estimators': 927}. Best is trial 78 with value: 0.8801458176072536.
[I 2024-07-16 08:50:02,649] Trial 298 finished with value: 0.8780239983129441 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.7439153716980458, 'loss': 'exponential', 'n_estimators': 978}. Best is trial 78 with value: 0.8801458176072536.
[I 2024-07-16 08:50:28,392] Trial 299 finished with value: 0.8747983038326094 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.8349636672105378, 'loss': 'square', 'n_estimators': 901}. Best is trial 78 with value: 0.8801458176072536.
[I 2024-07-16 08:50:55,737] Trial 300 finished with value: 0.8785160859016511 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.8577955998125565, 'loss': 'linear', 'n_estimators': 927}. Best is trial 78 with value: 0.8801458176

In [28]:
print(study_xgb.best_value)
print(study_xgb.best_params)

0.951622985121114
{'scaling_method': 'minmax', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 979, 'learning_rate': 0.046054720840163293, 'max_depth': 4, 'subsample': 0.8282908794962028, 'colsample_bytree': 0.5510975070017484, 'min_child_weight': 10}


In [ ]:
trial_number = []
score = []
parameters = []
optuna_dict = {}
trials = range(200)

for trial in trials:
    trial_number.append(study_lasso.get_trials()[trial].number)
    score.append(study_lasso.get_trials()[trial].value)
    parameters.append(list(study_lasso.get_trials()[trial].params.items()))

optuna_dict['Trial'] = trial_number
optuna_dict['Score'] = score
optuna_dict['Parameters'] = parameters

optuna_lasso = pd.DataFrame.from_dict(optuna_dict)
optuna_lasso

In [ ]:
fig = px.scatter(optuna_lasso,
                 x="Trial", 
                 y="Score", 
                 hover_data=['Parameters'],
                labels = {'Trial': 'Optuna Trial', 'Score': 'Score (R-Squared)'},
                title = 'Lasso tuned with Optuna after 200 trials  <br><sup> Best Score: 0.9508</sup>')

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)

fig.show()

# RF

In [37]:
def instantiate_rf(trial : Trial) -> RandomForestRegressor:
    params = {
    'bootstrap':trial.suggest_categorical('bootstrap', [True]),
    'n_estimators': trial.suggest_int('n_estimators', 250, 500),
    'max_depth': trial.suggest_int('max_depth', 50, 75),
    'min_samples_split': trial.suggest_int('min_samples_split', 4, 6),
    'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5)
    }
    
    return RandomForestRegressor(**params)
  
def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor_ordinal(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_rf(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_rf(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [38]:
from optuna import create_study

study_rf = create_study(study_name='optimization', direction='maximize')

#study_rf.optimize(lambda trial: objective_rf(trial, X, y), n_trials=5)

[I 2024-07-11 09:43:39,468] A new study created in memory with name: optimization
[I 2024-07-11 09:44:37,413] Trial 0 finished with value: 0.9083545094739573 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 253, 'max_depth': 60, 'min_samples_split': 5, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.9083545094739573.
[I 2024-07-11 09:45:24,724] Trial 1 finished with value: 0.8999706968816368 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 328, 'max_depth': 68, 'min_samples_split': 6, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.9083545094739573.
[I 2024-07-11 09:46:51,911] Trial 2 finished with value: 0.9081071301156255 and parameters: {'scaling_method': 'minmax', 'bootstrap': True, 'n_estimators': 427, 'max_depth': 73, 'min_samples_split': 5, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.9083545094739573.
[I 2024-07-11 09:48:22,636] Trial 3 finished with value: 0.9093564157184439 and parameters: {'sc

In [51]:
print(study_rf.best_value)
print(study_rf.best_params)

0.9106961840498735
{'scaling_method': 'robust', 'encoding_method': 'ordinal', 'bootstrap': True, 'n_estimators': 338, 'max_depth': 73, 'min_sample_split': 4, 'min_sample_leaf': 2}


# gbr

In [31]:
def instantiate_gbr(trial : Trial) -> GradientBoostingRegressor:
    params = {
    'n_estimators': trial.suggest_int('n_estimators', 500, 1000),
    'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1, log=True),
    'max_depth': trial.suggest_int ('max_depth', 2, 6),
    "subsample": trial.suggest_float("subsample", 0.5, 1.0),
    'min_samples_split': trial.suggest_int ('min_sample_split', 6, 10),
    'min_samples_leaf': trial.suggest_int ('min_sample_leaf', 1, 10)
    }
    
    return GradientBoostingRegressor(**params)

def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor_ordinal(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_gbr(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_gbr(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [34]:
from optuna import create_study

study_gbr = create_study(study_name='optimization', direction='maximize')

study_gbr.optimize(lambda trial: objective_gbr(trial, X, y), n_trials=5)

[I 2024-07-11 09:36:55,471] A new study created in memory with name: optimization
[I 2024-07-11 09:38:00,789] Trial 0 finished with value: 0.10676969273467858 and parameters: {'scaling_method': 'maxabs', 'n_estimators': 680, 'learning_rate': 0.00015291122193602163, 'max_depth': 6, 'subsample': 0.7453028457760895, 'min_sample_split': 10, 'min_sample_leaf': 9}. Best is trial 0 with value: 0.10676969273467858.
[I 2024-07-11 09:38:22,650] Trial 1 finished with value: 0.28100365000054095 and parameters: {'scaling_method': 'standard', 'n_estimators': 605, 'learning_rate': 0.0006146473911796216, 'max_depth': 2, 'subsample': 0.8742574967407413, 'min_sample_split': 8, 'min_sample_leaf': 2}. Best is trial 1 with value: 0.28100365000054095.
[I 2024-07-11 09:39:24,934] Trial 2 finished with value: 0.2466584750875623 and parameters: {'scaling_method': 'standard', 'n_estimators': 533, 'learning_rate': 0.00043180685670930226, 'max_depth': 6, 'subsample': 0.8638913349214837, 'min_sample_split': 8, 'mi

In [35]:
print(study_gbr.best_value)
print(study_gbr.best_params)

0.9214824552885755
{'scaling_method': 'robust', 'n_estimators': 868, 'learning_rate': 0.005429634831521693, 'max_depth': 3, 'subsample': 0.8347093949409654, 'min_sample_split': 10, 'min_sample_leaf': 10}


# Adaboost

In [53]:
def instantiate_ada(trial : Trial) -> AdaBoostRegressor:
    params = {
    'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1, log=True),
    'loss': trial.suggest_categorical('loss',['linear', 'square', 'exponential']),
    'n_estimators': trial.suggest_int('n_estimators', 1, 1000)
    }
    
    return AdaBoostRegressor(**params)

def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor_ordinal(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_ada(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_ada(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [54]:
from optuna import create_study

study_ada = create_study(study_name='optimization', direction='maximize')

study_ada.optimize(lambda trial: objective_ada(trial, X, y), n_trials=50)

[I 2024-07-11 10:02:52,147] A new study created in memory with name: optimization
[I 2024-07-11 10:02:52,518] Trial 0 finished with value: 0.7697034560200977 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.5591223026880627, 'loss': 'linear', 'n_estimators': 5}. Best is trial 0 with value: 0.7697034560200977.
[I 2024-07-11 10:03:10,819] Trial 1 finished with value: 0.8546529541652251 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.06846055794485856, 'loss': 'linear', 'n_estimators': 404}. Best is trial 1 with value: 0.8546529541652251.
[I 2024-07-11 10:03:29,877] Trial 2 finished with value: 0.875164826045092 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.5252293433374021, 'loss': 'exponential', 'n_estimators': 551}. Best is trial 2 with value: 0.875164826045092.
[I 2024-07-11 10:03:51,103] Trial 3 finished with value: 0.7894060614839619 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.006602759458806035, 'loss': 'square'

[I 2024-07-11 10:14:44,856] Trial 33 finished with value: 0.866576708984014 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.09929974514686613, 'loss': 'exponential', 'n_estimators': 581}. Best is trial 10 with value: 0.8780826782483189.
[I 2024-07-11 10:14:58,969] Trial 34 finished with value: 0.8703748675462952 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.2245325690083747, 'loss': 'exponential', 'n_estimators': 368}. Best is trial 10 with value: 0.8780826782483189.
[I 2024-07-11 10:15:17,983] Trial 35 finished with value: 0.876282939613254 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.5987616594370728, 'loss': 'exponential', 'n_estimators': 614}. Best is trial 10 with value: 0.8780826782483189.
[I 2024-07-11 10:15:40,207] Trial 36 finished with value: 0.8102956663214451 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.013357794606257661, 'loss': 'exponential', 'n_estimators': 454}. Best is trial 10 with value: 0.878

# Loop through?

In [61]:
studies = [study_lasso, study_ridge, study_en, study_xgb, study_rf, study_gbr, study_ada]
objectives = [objective_lasso, 
              objective_ridge, 
              objective_en,
              objective_xgb,
              objective_rf,
              objective_gbr,
              objective_ada]

# for study, objective in zip(studies, objectives):
#     study.optimize(lambda trial: objective(trial, X, y), n_trials=2)
    

[<function __main__.objective_lasso(trial: optuna.trial._trial.Trial, X: pandas.core.frame.DataFrame, y: numpy.ndarray | pandas.core.series.Series, numerical_columns: Optional[list[str]] = None, categorical_columns: Optional[list[str]] = None, random_state: int = 42) -> float>,
 <function __main__.objective_ridge(trial: optuna.trial._trial.Trial, X: pandas.core.frame.DataFrame, y: numpy.ndarray | pandas.core.series.Series, numerical_columns: Optional[list[str]] = None, categorical_columns: Optional[list[str]] = None, random_state: int = 42) -> float>,
 <function __main__.objective_en(trial: optuna.trial._trial.Trial, X: pandas.core.frame.DataFrame, y: numpy.ndarray | pandas.core.series.Series, numerical_columns: Optional[list[str]] = None, categorical_columns: Optional[list[str]] = None, random_state: int = 42) -> float>,
 <function __main__.objective_xgb(trial: optuna.trial._trial.Trial, X: pandas.core.frame.DataFrame, y: numpy.ndarray | pandas.core.series.Series, numerical_columns: O

In [52]:
for study in studies:
    print (study.best_value)
    print (study.best_params)

0.8735606638813536
{'scaling_method': 'robust', 'learning_rate': 0.30065504792403025, 'loss': 'exponential', 'n_estimators': 428}
0.7980960053211354
{'scaling_method': 'robust', 'learning_rate': 0.006736329679908936, 'loss': 'exponential', 'n_estimators': 615}
0.8177296198440576
{'scaling_method': 'maxabs', 'learning_rate': 0.04083092875953665, 'loss': 'exponential', 'n_estimators': 196}
0.951622985121114
{'scaling_method': 'minmax', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 979, 'learning_rate': 0.046054720840163293, 'max_depth': 4, 'subsample': 0.8282908794962028, 'colsample_bytree': 0.5510975070017484, 'min_child_weight': 10}
0.9094102521730991
{'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 494, 'max_depth': 62, 'min_samples_split': 4, 'min_samples_leaf': 1}
0.9214824552885755
{'scaling_method': 'robust', 'n_estimators': 868, 'learning_rate': 0.005429634831521693, 'max_depth': 3, 'subsample': 0.8347093949409654, 'min_sample_split': 10, 'min_sample_l